In [1]:
#!pip install bokeh on a standard python install
from bokeh.layouts import row,column
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook,push_notebook
from microtsetlin import *
import time
output_notebook()

Loading BokehJS ...

## Microtsetlin

This notebook contains WIP visualisation of tsetlin states using hinton diagrams - it's not completely self-explanatory
as there's no iteration variables to explain where we are in the trianing cycle

TODO:
1. output iteration
2. output input and desired out put

In [2]:
def hinton(f, matrix, thres,max_weight=105):
    if not max_weight:
        max_weight = 2 ** np.ceil(np.log(np.abs(matrix).max()) / np.log(2))
    out = [(y,x,w / max_weight, "black" if w > thres else "white") for (x, y), w in np.ndenumerate(matrix)]
    x,y,s,c = zip(*out)
    h = f.rect(x, y, fill_color=list(c),line_color="black", width=s, height=s)
    return f,h 

def uhinton(target, h,matrix, thres, max_weight=105):
    if not max_weight:
        max_weight = 2 ** np.ceil(np.log(np.abs(matrix).max()) / np.log(2))
    out = [(y,x,w/float(max_weight), "black" if w > thres else "white") for (x, y), w in np.ndenumerate(matrix)]
    x,y,s,c = zip(*out)
    h.data_source.data['x']= x
    h.data_source.data['y']= y
    h.data_source.data['width']= s
    h.data_source.data['height']= s
    h.data_source.data['fill_color']= list(c)
    push_notebook(handle=target)

In [7]:
a = ['E','I']
f = figure(title="Include", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500)
f,h = hinton(f,tsetlin[:,:,0],MIDSTATE)
f.xaxis.visible = False 
f.yaxis.visible = False
f2 = figure(title="Include Negative", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500)
f2,h2 = hinton(f2,tsetlin[:,:,1],MIDSTATE)
f2.xaxis.visible = False
f2.yaxis.visible = False
f3 = figure(title="Out", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500)
f3,h3 = hinton(f3,np.array(num_clauses*[0]).reshape(1, num_clauses),0.5)
f3.xaxis.visible = False 
f3.yaxis.visible = False

# f4 = figure(title="Desired/Acual", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500)
# f4,h4 = hinton(f4,np.array([0]+[Y[0]]).reshape(1, 2),0.5)
# f4.xaxis.visible = False 
# f4.yaxis.visible = False



In [8]:
target = show(row(f,f2,f3), notebook_handle=True)
tsetlin       = np.random.choice([MIDSTATE, MIDSTATE+1], size=(num_clauses,num_features, len([POS_LIT,NOT_LIT])))  

def train_vis():
    # print(np.transpose(tsetlin))
    for i in range(iterations):
        e = 0
        out = []
        for x_hat,y_hat in zip(X,Y):
            clause_output = calc_all_clause_outputs(x_hat, tsetlin)
            tot_c_outputs = np.sum([c_out*sign if live else 0 for (c_out,live),sign in zip(clause_output,clause_sign)])
            out += [tot_c_outputs] 
            y_est = 1 if tot_c_outputs >= 0 else 0 #nte: total 0 = output 1
            e += 1 if y_est != y_hat else 0
            update(tot_c_outputs,y_hat,polarity,clause_output,x_hat,tsetlin)
            uhinton(target,h,tsetlin[:,:,0],  MIDSTATE)
            uhinton(target,h2,tsetlin[:,:,1], MIDSTATE)
            actual_c,_ = zip(*clause_output)
            uhinton(target,h3,100*np.array(actual_c).reshape(num_clauses,1), 0.5)
            time.sleep(0.1)
            #uhinton(target,h4,np.array([100*y_est]+[100*y_hat]).reshape(1,2), 0.5)
    
train_vis()    

KeyboardInterrupt: 